![imagen con dos cerebros enfrentados, el de la izquierda azul y el de la derecha naranja de los que salen ramificaciones](../img/img_cabecera.PNG)

# Limpieza y transformaciones de los datos sobre el comportamiento de personas introvertidas y extrovertidas

Para poder continuar con el análisis, es necesario realizar una serie de cambios que nos faciliten conseguir los resultados esperados.

La exploración inicial se ha realizado en el archivo de ["exploración"](exploracion.ipynb).

La estructura a seguir es la siguiente:

- [0. Importación de librerías y carga de datos](#0-importación-librerías-y-carga-de-datos)
- [1. Transformaciones](#1transformaciones)
- [2. Limpieza de valores nulos](#2-limpieza-de-valores-nulos)
    - [2.1 Variables categóricas](#21-variables-categóricas)
    - [2.2 Variables numéricas](#22-variables-numéricas)
- [3. Exportación del csv transformado](#3-exportación-del-csv-transformado)



### 0. Importación librerías y carga de datos

In [1]:
# Librerias necesarias:
import pandas as pd
import numpy as np
import sys
sys.path.append("..")

import src.transformaciones as tr

Funciones ejecutadas correctamente


In [2]:
# Carga del csv:
df = pd.read_csv('../data/personality_dataset.csv')

### 1.Transformaciones

El primer paso de limpieza es normalizar los títulos de las columnas convirtiéndolos todos a minúsculas. Esto evitará errores al referenciar columnas en futuras operaciones.

In [3]:
df.columns = df.columns.str.lower()
print(df.columns)

Index(['time_spent_alone', 'stage_fear', 'social_event_attendance',
       'going_outside', 'drained_after_socializing', 'friends_circle_size',
       'post_frequency', 'personality'],
      dtype='object')


Se ejecuta la función *clasificador_variables* para obtener los nombres de las columnas modificados y realizar correctamente los siguientes pasos del análisis.

In [4]:
target, numericas, categoricas = tr.clasificador_variables(df, 'personality')

print(f'Columna objetivo: {target}')
print(f'Columnas numéricas: {numericas}')
print(f'Columnas categóricas: {categoricas}')


Columna objetivo: personality
Columnas numéricas: ['time_spent_alone', 'social_event_attendance', 'going_outside', 'friends_circle_size', 'post_frequency']
Columnas categóricas: ['stage_fear', 'drained_after_socializing']


A continuación se cuentan los valores de cada variable categórica. Esto nos permitirá ver la necesidad de normalizar los datos así como la existencia de valores únicos. Como la variable objetivo también presenta valores de este tipo, se crea una variable nueva para facilitar el análisis.

In [5]:
target_categoricas = [target] + categoricas

print(target_categoricas)

['personality', 'stage_fear', 'drained_after_socializing']


In [6]:
tr.contar_valores_categoricos(df,target_categoricas)

Columna personality:
personality
Extrovert    1491
Introvert    1409
Name: count, dtype: int64


Columna stage_fear:
stage_fear
No     1417
Yes    1410
NaN      73
Name: count, dtype: int64


Columna drained_after_socializing:
drained_after_socializing
No     1441
Yes    1407
NaN      52
Name: count, dtype: int64




En cuanto a las transformaciones de estas columnas, para noralizar los datos, también es recomendable pasarlo todo a minúscula y eliminar cualquier posible espacio que haya delante o detrás de los valores. Otro cambio que sería necesario para aplicar modelos predictivos sería pasar todo a 0 y 1. Sin embargo, para que las visualizaciones posteriores se vean con mayor claridad omitiremos este paso.

In [7]:
df_2 = tr.normalizar_valores_categoricos(df,target_categoricas)

In [8]:
df_2.head()

,time_spent_alone,stage_fear,social_event_attendance,going_outside,drained_after_socializing,friends_circle_size,post_frequency,personality
0,4.0,no,4.0,6.0,no,13.0,5.0,extrovert
1,9.0,yes,0.0,0.0,yes,0.0,3.0,introvert
2,9.0,yes,1.0,2.0,yes,5.0,2.0,introvert
3,0.0,no,6.0,7.0,no,14.0,8.0,extrovert
4,3.0,no,9.0,4.0,no,8.0,5.0,extrovert


### 2. Limpieza de valores nulos

#### 2.1 Variables categóricas

Una vez normalizados todos los datos, procedemos a ver los nulos en detalle para decidir por qué valor vamos a imputarlo. Para optimizar el código usaremos la función definida *contar_nulos*.

In [9]:
tr.contar_nulos(df_2,categoricas)

Valores nulos de stage_fear = 73
Valores nulos de drained_after_socializing = 52


Para ambas categorías se decide agruparlas según la columna objetivo y obtener así la distribución. A rasgos generales, las personas que tienen pánico escénico suelen ser introvertidas así como aquellas que se sienten agotadas después de socializar.

In [10]:
df_2.groupby('personality')['stage_fear'].value_counts(dropna = False)

personality  stage_fear
extrovert    no            1338
             yes            111
             NaN             42
introvert    yes           1299
             no              79
             NaN             31
Name: count, dtype: int64

In [11]:
df_2.groupby('personality')['drained_after_socializing'].value_counts(dropna = False)

personality  drained_after_socializing
extrovert    no                           1362
             yes                           111
             NaN                            18
introvert    yes                          1296
             no                             79
             NaN                            34
Name: count, dtype: int64

La siguiente cuestión que se plantea es si todas las personas que tienen pánico escénico se sienten agotadas al socializar y viceversa. Por lo que se agrupan ambas columnas en función de la personalidad.
Los valores obtenidos nos permiten confirmar que todos los estudiantes que se sienten agotados despues de socializar tienen pánico escénico y que todos aquellos que no se sienten agotados al socializar no lo tienen.

In [12]:
df_2.groupby(['drained_after_socializing','stage_fear'])['personality'].value_counts()

drained_after_socializing  stage_fear  personality
no                         no          extrovert      1320
                                       introvert        79
yes                        yes         introvert      1266
                                       extrovert       111
Name: count, dtype: int64

In [13]:
df_2[(df_2['stage_fear'] == 'Yes')&(df_2['drained_after_socializing'] == 'No')]

,time_spent_alone,stage_fear,social_event_attendance,going_outside,drained_after_socializing,friends_circle_size,post_frequency,personality


Antes de proceder a las transformaciones, vemos como se distribuyen los nulos dentro de la columna *drained_after_socializing* en función de los valores de la columna *stage_fear*. De este modo, se observa que la mayoría de ellos corresponden a aquellos que presentan pánico escénico.

In [14]:
stage_yes = df_2[(df_2['stage_fear'] == 'yes') & (df_2['drained_after_socializing'].isna())].shape[0]
stage_no = df_2[(df_2['stage_fear'] == 'no') & (df_2['drained_after_socializing'].isna())].shape[0]

print(f'Si "stage_fear" es "yes" hay {stage_yes} nulos de la columna "drained_after_socializing".')
print(f'Si "stage_fear" es "no" hay {stage_no} nulos de la columna "drained_after_socializing".')

Si "stage_fear" es "yes" hay 33 nulos de la columna "drained_after_socializing".
Si "stage_fear" es "no" hay 18 nulos de la columna "drained_after_socializing".


Aplicando la misma lógica, observamos que la mayoría de nulos también se encuentran en aquellos que se sienten agotados después de socializar.

In [15]:
drained_yes = df_2[(df_2['drained_after_socializing'] == 'yes') & (df_2['stage_fear'].isna())].shape[0]
drained_no = df_2[(df_2['drained_after_socializing'] == 'no') & (df_2['stage_fear'].isna())].shape[0]

print(f'Si "drained_after_socializing" es "yes" hay {drained_yes} nulos de la columna "stage_fear".')
print(f'Si "drained_after_socializing" es "no" hay {drained_no} nulos de la columna "stage_fear".')

Si "drained_after_socializing" es "yes" hay 30 nulos de la columna "stage_fear".
Si "drained_after_socializing" es "no" hay 42 nulos de la columna "stage_fear".


A pesar de que por esta lógica exista se podrían imputar los valores nulos, la decisión tomada es eliminar las filas ya que no superan el 3% de los datos.

#### 2.2 Variables numéricas

Al igual que con las variables categóricas es necesario observar sus valores estadísticos en función de la variable objetivo para decidir cual es la que se podría imputar en los valores nulos.

En primer lugar observamos como la media y la mediana coinciden en todas las variables, lo que indica que al haberlos agrupados por *personality* conseguimos valores más homogéneos. Sin embargo, sí que existe diferencia entre la moda y estos valores ya que en datasets como este, que contiene escalas discretas y ordinales, es muy común que estos valores no coinciden.

In [16]:
tr.comparativa_media_mediana_moda(df_2,numericas,target)

Media de time_spent_alone según personality:
personality
extrovert    2.0
introvert    7.0
Name: time_spent_alone, dtype: float64


Mediana de time_spent_alone según personality:
personality
extrovert    2.0
introvert    7.0
Name: time_spent_alone, dtype: float64


Moda de time_spent_alone según personality:
personality
extrovert    3.0
introvert    9.0
Name: time_spent_alone, dtype: float64

---------------------------------------------------------

Media de social_event_attendance según personality:
personality
extrovert    6.0
introvert    2.0
Name: social_event_attendance, dtype: float64


Mediana de social_event_attendance según personality:
personality
extrovert    6.0
introvert    2.0
Name: social_event_attendance, dtype: float64


Moda de social_event_attendance según personality:
personality
extrovert    4.0
introvert    2.0
Name: social_event_attendance, dtype: float64

---------------------------------------------------------

Media de going_outside según personality:
person

Como en el caso anterior, estas variables tamposo superan el 3% de los nulos. Por lo tanto, se eliminarán todas las filas con nulos, dejando un totoal de 2477 filas de las 2900 originales.

In [20]:
df_2.isna().sum()

time_spent_alone             63
stage_fear                   73
social_event_attendance      62
going_outside                66
drained_after_socializing    52
friends_circle_size          77
post_frequency               65
personality                   0
dtype: int64

In [21]:
df_clean = df_2.dropna()

In [22]:
df_clean.shape

(2477, 8)

In [24]:
tr.comparativa_media_mediana_moda(df_clean,numericas,target)

Media de time_spent_alone según personality:
personality
extrovert    2.0
introvert    7.0
Name: time_spent_alone, dtype: float64


Mediana de time_spent_alone según personality:
personality
extrovert    2.0
introvert    7.0
Name: time_spent_alone, dtype: float64


Moda de time_spent_alone según personality:
personality
extrovert    0.0
introvert    9.0
Name: time_spent_alone, dtype: float64

---------------------------------------------------------

Media de social_event_attendance según personality:
personality
extrovert    6.0
introvert    2.0
Name: social_event_attendance, dtype: float64


Mediana de social_event_attendance según personality:
personality
extrovert    6.0
introvert    2.0
Name: social_event_attendance, dtype: float64


Moda de social_event_attendance según personality:
personality
extrovert    4.0
introvert    2.0
Name: social_event_attendance, dtype: float64

---------------------------------------------------------

Media de going_outside según personality:
person

### 3. Exportación del csv transformado

In [26]:
df_clean.to_csv("../data/data_transformed_no_nul.csv",index = False)